In [65]:
import pandas as pd
import requests
import urllib.parse
from IPython.display import display

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [3]:
def validate_urls(src_url, tgt_url):
    print(f"src_url:{src_url} \ntgt_url:{tgt_url}")
    assert src_url.netloc != tgt_url.netloc
    assert src_url.path == tgt_url.path
    assert src_url.params == tgt_url.params
    assert src_url.query == tgt_url.query
    assert src_url.fragment == tgt_url.fragment

In [67]:
def fetch_data(src_url, tgt_url):
    src_json = requests.get(
        src_url.geturl(), 
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=15,
    )
    src_df = pd.DataFrame(src_json.json()['result'])
    
    tgt_json = requests.get(
        tgt_url.geturl(), 
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=15,
    )
    tgt_df = pd.DataFrame(tgt_json.json()['result'])

    return (src_df, tgt_df)

In [56]:
def clean_df(df, excluded_cols):
    colFilter = df.filter(excluded_cols)
    df = df.drop(colFilter, axis=1)
    if 'fid' in df.columns:
        df['fid'] = df['fid'].astype("Int32")
    return df

In [46]:
def compare_data(src_df, tgt_df, excluded_cols):
    src_df = clean_df(src_df, excluded_cols)
    print("***** Source Data Sample:")
    display(src_df.sample())

    tgt_df = clean_df(tgt_df, excluded_cols)
    print("***** Target Data Sample:")
    display(tgt_df.sample())
    
    cmp_df = src_df.compare(tgt_df, result_names=('src', 'tgt'), align_axis=1)
    cmp_df = clean_df(cmp_df, excluded_cols)
    print("***** Comparison Data Sample:")
    display(cmp_df.sample())
    
    return (src_df, tgt_df, cmp_df)

In [47]:
def validate_fetch_compare(src_url, tgt_url):
    validate_urls(src_url, tgt_url)
    (src_df, tgt_df) = fetch_data(src_url, tgt_url)
    return compare_data(src_df, tgt_df, ['bal_update_ts'])

# Channels

In [8]:
channel_id = input()

 farcaster


## Stats

In [48]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/rankings/{channel_id}/stats?rank_timeframe=60d")

In [49]:
tgt_url = urllib.parse.urlparse(f"https://graph.cast9.k3l.io/channels/rankings/{channel_id}/stats?rank_timeframe=60d")

In [50]:
(_, _, comparison) = validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/rankings/farcaster/stats', params='', query='rank_timeframe=60d', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast9.k3l.io', path='/channels/rankings/farcaster/stats', params='', query='rank_timeframe=60d', fragment='')
***** Source Data Sample:


,channel_id,num_followers,num_fids_ranked,min_score,max_score,num_members,num_holders,min_balance,max_balance,token_num_holders,token_min_balance,token_max_balance
0,farcaster,386554,13499,0.00,0.09,316,14609,0,"60,855.68",None,None,None


***** Target Data Sample:


,channel_id,num_followers,num_fids_ranked,min_score,max_score,num_members,num_holders,min_balance,max_balance,token_num_holders,token_min_balance,token_max_balance
0,farcaster,356391,12621,0.00,0.09,316,14597,0,"60,850.68",None,None,None


***** Comparison Data Sample:


num_followers         num_fids_ranked                   min_score                                 max_score                      num_holders                 max_balance                     
            src     tgt             src    tgt                  src                  tgt                  src                  tgt         src    tgt                  src                  tgt
0        386554  356391           13499  12621                 0.00                 0.00                 0.09                 0.09       14609  14597            60,855.68            60,850.68

## Rankings

In [51]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/rankings/{channel_id}"
                                "?rank_timeframe=60d&offset=0&limit=1000&lite=true")
tgt_url = urllib.parse.urlparse(f"https://graph.cast9.k3l.io/channels/rankings/{channel_id}"
                                "?rank_timeframe=60d&offset=0&limit=1000&lite=true")

In [52]:
(_, _, comparison) = validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/rankings/farcaster', params='', query='rank_timeframe=60d&offset=0&limit=1000&lite=true', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast9.k3l.io', path='/channels/rankings/farcaster', params='', query='rank_timeframe=60d&offset=0&limit=1000&lite=true', fragment='')
***** Source Data Sample:


,fid,rank
961,665738,962


***** Target Data Sample:


,fid,rank
733,2684,734


***** Comparison Data Sample:


fid        
        src     tgt
136  256931  535208

In [53]:
display(comparison)

fid         
         src      tgt
12      2802     4823
13      4823     2802
14      4753     8451
15      2270      239
16       239     2270
17       733     4753
18    195117      733
19    279477   195117
20      5395   279477
21    253730     5395
22    243913   253730
24    212342   243913
25      2242   212342
26      1628     2242
29       341     1628
30      2982      341
31      4923     2982
32      3621     4923
36     16098     8152
37      3635      114
38      8152     3621
39       114     4179
41      8451     2745
43      4179    16098
44      1734      347
45      6023     1689
46       725     6023
47      2745     3635
48       347    13659
49     13659     4895
50    194339      725
51    234616    12480
52      1689   375245
53     12480    19129
54    375245   234616
55     19129    17555
56     17555      206
57    247143   846335
58       142     4368
59     11517     3887
60      3887   190108
61       206   240586
62    846335   246221
63    240586      431
64    535199     1725
65      4368     6196
66      4895   420493
67       431   753612
68    509624     7692
69    243300      453
70      1725    12777
72    246221   535199
73    420493   391793
74      6196   281836
75    391793     2902
76      2902     1082
77    753612      194
78       768   247143
79       194       12
80      7692    11517
81       453   382802
82      1082     1898
83     12777     1795
84    281836     9135
85      4327   509624
86      1795   547241
87      5818     5774
88      9702      142
89      4282     9702
90      4085       56
91      9135   685132
92    382802     4085
93        12     4282
94      2007   243300
95      1898   296687
96      5774     5818
97    547241      738
98    321969  1017391
99      8447     6806
100      738      260
101     6591     7097
102     2099     2341
103       56      880
104     6806      533
105     7097   862100
106   296687    12418
107   685132   639226
108      533     1355
109  1017391    13874
110    13874   280220
111      260   525471
112      880    11528
113    12418     6373
114     1355   283056
115     4167   194372
116   862100     4167
117   639226   263367
118   194372   982306
119   510796     2007
120    11528   967595
121     4268   321969
122   283056    18723
123   967595     6591
124   525471   268438
125     6373     1356
126   280220     1734
127   982306     4327
128     2341   995449
129     5494   510796
130    18723   337090
131    12224     5049
132   337090    13833
133     1356   977233
134   268438     3854
135   263367   256931
136   256931   535208
137    16567    11948
138   977233     1656
139   204221     8447
140     1656     4407
141     3854   204221
142   995449      274
143   535208      616
144     5049   193822
145    11948    19627
146     5406     6131
147      274     2723
148      616     1020
149     4407     1407
150      359     9326
151    19627     1886
152    13833      359
153   193822   864405
154     5964   342680
155     4294      358
156     1020   409857
157     6131     6245
158     2723     5494
159   864405    12256
160   446697   323251
161   409857    18138
162     1886    12142
163   342680    11124
164    12142     8476
165   323251    16567
166    16148    20066
167    18138   354255
168    12256   326567
169   329898      922
170     1407     4294
171      358   188926
172    11124   872898
173   354255     5309
174   326567   270504
175   191294    12224
176     8476       66
177   301340    16148
178   270504   196842
179    20066     5713
180   190108     5964
181   415337   301340
182     4479   886870
183      922  1015455
184     6631    15900
185   188926     4389
186      602     1626
187   872898   982331
188   886870   192539
189   196842     4268
190     1626      602
191     5309     8637
192     5713   270170
193     7479      768
194   214447   514366
195     3265   646397
196       66   190218
197     2480     7479
198   190218     5406
199     8637     6111
200  1015455   262970
201  

## Holders

In [54]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/holders/{channel_id}"
                                "?rank_timeframe=60d&orderby=total&offset=0&limit=100")
tgt_url = urllib.parse.urlparse(f"https://graph.cast9.k3l.io/channels/holders/{channel_id}"
                                "?rank_timeframe=60d&orderby=total&offset=0&limit=100")

In [57]:
(src_df, tgt_df, cmp_df) = validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/holders/farcaster', params='', query='rank_timeframe=60d&orderby=total&offset=0&limit=100', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast9.k3l.io', path='/channels/holders/farcaster', params='', query='rank_timeframe=60d&orderby=total&offset=0&limit=100', fragment='')
***** Source Data Sample:


,top_ptile,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,is_points_launched,is_tokens_launched,memberat,followedat,approx_fid_originat,order_rank
76,1,431,j4ck.eth,j4ck.eth,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/3b82cfba-7d62-43ad-f752-1643db239b00/original,CEO at Google • co-founder & rectangle mover at /icebreaker • icbk.me/j4ck.fc • /j4ck • /swagcaster • DJ sixsix • MMTF,farcaster,68.00,0.00,94,"[0x40319e404c6a269888869d47ddc8940ea29758c94c51d6dfd1cfb31fb7d0d40d, 0x633e9ec9fa22f749d2456eebffd12ba37f9be07a, 0xdfe8beee223412f316baf2968b17527d6eba29f1]","1,032.53",None,0.00,0,True,None,None,0,0,True,False,"1,725,151,160.00","1,691,551,015.00","1,694,092,619.00",77


***** Target Data Sample:


,top_ptile,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,is_points_launched,is_tokens_launched,memberat,followedat,approx_fid_originat,order_rank
64,1,12480,thumbsup,thumbsup.eth,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/8c3811c6-6825-4719-0bfc-7a1bb50a5900/rectcrop3,writing stuff about tech + decentralization,farcaster,52.00,0.00,239,"[0x355423cb2017156f74223cc414251a58007b6136, 0x4d85e4f760fb58e380f02657ae5aafb8bd010601, 0x89cb2cccfd4876f256242913307e4762e21ce135, 0xd6c113eddfffaf2e54e76960e8b719cdf01cd470140bbc8794a63bc6e4316433]","1,195.39",None,0.00,0,True,None,None,0,0,True,False,"1,725,179,276.00","1,694,232,532.00","1,719,407,380.00",65


***** Comparison Data Sample:


fid       fname      username       pfp       bio      rank                    score                      global_rank     addresses                   balance                      daily_earnings          weekly_earnings                      memberat     followedat     approx_fid_originat    
     src   tgt   src  tgt      src  tgt  src  tgt  src  tgt  src tgt                  src                  tgt         src tgt       src  tgt                  src                  tgt            src tgt                  src                  tgt      src tgt        src tgt                 src tgt
11  <NA>  <NA>   NaN  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN                 0.01                 0.01         NaN NaN       NaN  NaN             8,054.98             8,057.98            NaN NaN                86.00                88.00      NaN NaN        NaN NaN                 NaN NaN

In [58]:
display(cmp_df)

fid                        fname                                 username                                                                                                                                                                   pfp                                                                                                                                                                                                                                                                                                                  bio                                                                                                                                                                                     rank                                     score                               global_rank                                                                                                                                                                                                                                                                  addresses                                                                                                                                                                                                                        balance                            daily_earnings                           weekly_earnings                                  memberat                                followedat                       approx_fid_originat                     
       src     tgt                  src                 tgt                  src                  tgt                                                                                                                                              src                                                                                                                                              tgt                                                                                                                                                                 src                                                                                                                                                                 tgt                  src                  tgt                  src                  tgt                  src                  tgt                                                                                                                                                                                                                                                   src                                                                                                                                                                                                       tgt                  src                  tgt                  src                  tgt                  src                  tgt                  src                  tgt                  src                  tgt                  src                  tgt
0     <NA>    <NA>        danromero.eth                 dwr                  NaN                  NaN                                                                                                                                              NaN                                                                                                                                              NaN                                                                                                                                                                 NaN                                                                                                                                                                 NaN                  NaN                  NaN                 0.09                 0.09                  NaN                  NaN                                                                      

In [59]:
src_df.loc[[8, 13, 16]]

,top_ptile,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,is_points_launched,is_tokens_launched,memberat,followedat,approx_fid_originat,order_rank
8,1,16085,samuellhuber.eth,samuellhuber.eth,https://i.imgur.com/OHMozjv.jpg,"Farcaster Development Boutique @ https://dTech.vision 💜 in love with tech 🤓decentralization, fitness and oldschool rap 😎",farcaster,10.00,0.02,102,"[0x09cedb7bb69f9f6df646dba107d2baacda93d6c9, 0x4dc5c0384f06b740750b623b26beb46cbd4c4e00]","12,207.59",None,0.00,0,True,None,None,0,0,True,False,"1,725,186,708.00",NaN,"1,722,985,958.00",9
13,1,725,sahil,sahil,https://lh3.googleusercontent.com/ELQOEk4_UQ1iLnH3VBoNovAoYrEn2Fpm73fQb8cE1wsVCFLwU6eT3USqlb7P8Ju6JLBKacWt_otOsOHy0GrZOumx7xQk_i-42lO97A,onchain reputation at karma3labs and @openrank,farcaster,47.00,0.00,787,"[0x35516c8e94d3e852b193eae5b1022d8f099175e3, 0x4a12d8389696eff9294dece42a648588eda0f56d]","5,005.55",None,50.00,0,True,None,None,455,0,True,False,"1,725,152,676.00","1,696,905,231.00","1,678,862,552.00",14
16,1,2270,jalil.eth,jalil.eth,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/d3593749-07df-41de-bf48-3b5f029e8200/original,Growth is infinite. Work is worship.\n\n/vv /opepen /checks,farcaster,16.00,0.00,1898,"[0x26575d92accbf6f5b92c281e0f5f6a6613299345, 0xe11da9560b51f8918295edc5ab9c0a90e9ada20b]","3,515.15",None,0.00,0,True,None,None,0,0,True,False,NaN,"1,691,350,555.00","1,726,491,318.00",17


In [76]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/casts/popular/{channel_id}"
                                "?rank_timeframe=60d&offset=0&limit=100&lite=false&provider_metadata=%257B%2522feedType%2522%3A%2522trending%2522%2C%2522lookback%2522%3A%2522six_months%2522%257D")
tgt_url = urllib.parse.urlparse(f"https://graph.cast9.k3l.io/channels/casts/popular/{channel_id}"
                                "?rank_timeframe=60d&offset=0&limit=100&lite=false&provider_metadata=%257B%2522feedType%2522%3A%2522trending%2522%2C%2522lookback%2522%3A%2522six_months%2522%257D")

In [77]:
(src_df, tgt_df, cmp_df) = validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/casts/popular/farcaster', params='', query='rank_timeframe=60d&offset=0&limit=100&lite=false&provider_metadata=%257B%2522feedType%2522%3A%2522trending%2522%2C%2522lookback%2522%3A%2522six_months%2522%257D', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast9.k3l.io', path='/channels/casts/popular/farcaster', params='', query='rank_timeframe=60d&offset=0&limit=100&lite=false&provider_metadata=%257B%2522feedType%2522%3A%2522trending%2522%2C%2522lookback%2522%3A%2522six_months%2522%257D', fragment='')
***** Source Data Sample:


,cast_hash,fid,channel_id,channel_rank,global_rank,ptile,fname,username,pfp,bio,cast_score,reaction_count,age_hours,age_days,cast_ts,text
68,0xd448f6b1d47e60366736173c9348e29a1d8a616a,961093,farcaster,4328,233647,7,pepetrump,pepetrump,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/ac27d402-91a9-44c5-b7dd-86ba727a2500/rectcrop3,None,0.17,3,1,0,2025-03-19T20:25:14,i have idea you ft Launch token Milady10% because this is good ticker


***** Target Data Sample:


,cast_hash,fid,channel_id,channel_rank,global_rank,ptile,fname,username,pfp,bio,cast_score,reaction_count,age_hours,age_days,cast_ts,text
49,0x3388b0deeb5cd00c94566b10078f940ec7b94e15,296301,farcaster,997,30804,5,justin77,justin77,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/6565d7ee-d685-4ce1-dd03-3445bb5e8a00/rectcrop3,"I am crypto enthusiastic and artist \nlove crypto, NFTs, Art\n\n\n\nWeb3 researcher ☄️",0.23,14,14,0,2025-03-19T07:57:23,what do you think friends 😅 \n\nwhich topic should be more discussed on FARCASTER \n\ni choose crypto market \nand you 🫵🥹


***** Comparison Data Sample:


cast_hash                                                 fid         channel_rank               global_rank                                     ptile                           fname                    username                                                                                                               pfp                                                                                                                                  bio                                                                                                                                       cast_score                            reaction_count                                 age_hours                                   cast_ts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          text                                                                                                                                          
                                           src                                         tgt     src     tgt          src  tgt                  src                  tgt                  src                  tgt        src              tgt        src              tgt                                                                                              src                                                                                              tgt                                 src                                                                                                                         tgt                  src                  tgt                  src                  tgt                  src                  tgt                  src                  tgt                                                                                                                                                                                                                                                                                                                                                                                                                                                                      src                                                                                                                                       tgt
62  0xe0fee1660ea50ba1c7a273682a46c5d197451e81  0xabe12584e5348f8c964249a98cc4da6e06550939  607731  872568         1395  481             9,981.00             7,216.00                 6.00                 7.00  turabkhan  cryptfi-mariano  turabkhan  cryptfi-mariano  https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/b8d8c3c0-8d2d-4cd7-f05d-03f745b0b300/rectcrop3  https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/3ae75913-5d01-4593-1af9-849bb09e3900/rectcrop3  I am interested in Bitcoin trading  Web3 Creator + Blockchain writer ~ Bringing products onchain | bringing Africa Onchain @base | community manager /pronouns                 0.19                 0.18                 8.00                 3.00                14.00                 3.00  2025-03-19T07:29:22  2025-03-19T18:33:44  This collage beautifully captures the architectural grandeur of Lahore, Pakistan. Featuring iconic landmarks like the Lahore Fort, Minar-e-Pakistan, Shalimar Gardens, Quaid-e-Azam Library, Aitchison College, and the majestic Badshahi Mosque, the images reflect the city's rich history, cultural heritage, and Mughal-era splendor. The blend of historical and modern elements showcases Lahore's timeless beauty and its significance as the heart of Pakistan.  GE FC \n15 days and few hour

In [78]:
cmp_df

cast_hash                                                  fid          channel_rank                global_rank                                     ptile                                 fname                           username                                                                                                                                                                pfp                                                                                                                                                                                                                                                                                                                    bio                                                                                                                                                                                 cast_score                            reaction_count                                 age_hours                                   cast_ts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     text  \
                                           src                                         tgt      src      tgt          src   tgt                  src                  tgt                  src                  tgt              src              tgt               src               tgt                                                                                                                                              src                                                                                                                                              tgt                                                                                                                                                                   src                                                                                                                                                                   tgt                  src                  tgt                  src                  tgt                  src                  tgt                  src                  tgt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 src   
0                                          NaN                                         